In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main-2'

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)

test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape


# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000, 1])


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 5000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]

In [11]:
data_dir = './cnn-lstm-imdb-hyperband-trails_15/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./cnn-lstm-imdb-hyperband-trails_15/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [12]:
import torch
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,trial,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_size = trial.suggest_int("hidden_size",32,256)
        embed_size = trial.suggest_int("embed_size",16,128)
        filter_size = trial.suggest_int("filter_size",50,200)
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        #self.cnn = CNN(vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len)
        self.lstm = nn.LSTM(
            input_size=3*filter_size, 
            hidden_size=hidden_size, 
            num_layers=1,
            batch_first=True)
        self.dense = nn.Linear(hidden_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x):
        
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        # print("x1")
        # print(x1.shape)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        # print("f1")
        # print(f1.shape)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        c_out = torch.cat([f1,f2,f3],dim=1)
        r_in = c_out.view(batch_size,1, -1)
        lstm_out, _ = self.lstm(r_in)
        out = self.dense(lstm_out[:, -1, :])
        return self.sig(out)

In [13]:
import time
import numpy as np 
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Combine(trial,vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = optim.Adam(model.parameters())
    loss_fn = torch.nn.BCELoss()
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    val_acc = 0
    for epoch in range(epochs):
        start = time.time()
        model.train()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(loss.item())
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
            val_acc = np.round((correct/total)*100, 3)
    write_to_csv(trial.number,epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return val_acc


def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    print("Accuracy:", (correct/total)*100)

In [14]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [15]:
import torch.optim as optim
epochs = 10

In [16]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=15,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 21:10:26,650] A new study created in memory with name: no-name-20bc19e9-4151-4cde-b250-f2d788660fba


Epoch: 0, BCELoss: 0.6934993675351143
Val Loss: 0.693 	Val Acc: 0.508
Epoch: 1, BCELoss: 0.6933403845628102
Val Loss: 0.693 	Val Acc: 0.508
Epoch: 2, BCELoss: 0.5336921380956968
Val Loss: 0.409 	Val Acc: 0.823
Epoch: 3, BCELoss: 0.35764983142415685
Val Loss: 0.363 	Val Acc: 0.846
Epoch: 4, BCELoss: 0.30623595555623373
Val Loss: 0.336 	Val Acc: 0.859
Epoch: 5, BCELoss: 0.27704234308252734
Val Loss: 0.328 	Val Acc: 0.863
Epoch: 6, BCELoss: 0.25526158455759285
Val Loss: 0.348 	Val Acc: 0.849
Epoch: 7, BCELoss: 0.23996724975605807
Val Loss: 0.327 	Val Acc: 0.861
Epoch: 8, BCELoss: 0.2271840187907219
Val Loss: 0.353 	Val Acc: 0.848
Epoch: 9, BCELoss: 0.21623181816190482


[I 2022-05-14 21:11:46,766] Trial 0 finished with value: 86.37 and parameters: {'optimizer': 'SGD', 'lr': 0.0004730282867132955, 'dropout': 0.5649000155479039, 'hidden_size': 151, 'embed_size': 16, 'filter_size': 163}. Best is trial 0 with value: 86.37.


Val Loss: 0.323 	Val Acc: 0.864
Epoch: 0, BCELoss: 0.6929472298423449
Val Loss: 0.693 	Val Acc: 0.493
Epoch: 1, BCELoss: 0.46640263686577477
Val Loss: 0.378 	Val Acc: 0.834
Epoch: 2, BCELoss: 0.34290798505147296
Val Loss: 0.355 	Val Acc: 0.847
Epoch: 3, BCELoss: 0.2991393394768238
Val Loss: 0.339 	Val Acc: 0.853
Epoch: 4, BCELoss: 0.26743185809503
Val Loss: 0.323 	Val Acc: 0.862
Epoch: 5, BCELoss: 0.2464319201434652
Val Loss: 0.316 	Val Acc: 0.866
Epoch: 6, BCELoss: 0.23053405116622647
Val Loss: 0.313 	Val Acc: 0.869
Epoch: 7, BCELoss: 0.2150389173688988
Val Loss: 0.339 	Val Acc: 0.864
Epoch: 8, BCELoss: 0.2045970657405754
Val Loss: 0.338 	Val Acc: 0.861
Epoch: 9, BCELoss: 0.1934785515318314


[I 2022-05-14 21:13:18,508] Trial 1 finished with value: 85.1 and parameters: {'optimizer': 'Adam', 'lr': 0.0017489082807533743, 'dropout': 0.5692791082666415, 'hidden_size': 173, 'embed_size': 47, 'filter_size': 102}. Best is trial 0 with value: 86.37.


...

Val Loss: 0.322 	Val Acc: 0.876
Epoch: 0, BCELoss: 0.6600690307716529
Val Loss: 0.483 	Val Acc: 0.770
Epoch: 1, BCELoss: 0.38337157234549524
Val Loss: 0.337 	Val Acc: 0.857
Epoch: 2, BCELoss: 0.29947164477159577
Val Loss: 0.310 	Val Acc: 0.871
Epoch: 3, BCELoss: 0.261622343386213
Val Loss: 0.304 	Val Acc: 0.875
Epoch: 4, BCELoss: 0.23285146425788603
Val Loss: 0.308 	Val Acc: 0.876
Epoch: 5, BCELoss: 0.212929193880409
Val Loss: 0.309 	Val Acc: 0.878
Epoch: 6, BCELoss: 0.1970222249130408
Val Loss: 0.310 	Val Acc: 0.877
Epoch: 7, BCELoss: 0.18159213591056567
Val Loss: 0.329 	Val Acc: 0.875
Epoch: 8, BCELoss: 0.1641764525230974
Val Loss: 0.331 	Val Acc: 0.876
Epoch: 9, BCELoss: 0.15233335985802113


[I 2022-05-14 21:36:29,114] Trial 13 finished with value: 87.7 and parameters: {'optimizer': 'Adam', 'lr': 0.023971812037957868, 'dropout': 0.43852645869169377, 'hidden_size': 84, 'embed_size': 44, 'filter_size': 131}. Best is trial 2 with value: 87.81.


Val Loss: 0.354 	Val Acc: 0.877
Epoch: 0, BCELoss: 0.6513309746980667
Val Loss: 0.469 	Val Acc: 0.778
Epoch: 1, BCELoss: 0.380674097041289
Val Loss: 0.355 	Val Acc: 0.849
Epoch: 2, BCELoss: 0.2999256575728456
Val Loss: 0.322 	Val Acc: 0.866
Epoch: 3, BCELoss: 0.2549949782838424
Val Loss: 0.318 	Val Acc: 0.869
Epoch: 4, BCELoss: 0.22609017677605153
Val Loss: 0.326 	Val Acc: 0.873
Epoch: 5, BCELoss: 0.2017523917555809
Val Loss: 0.338 	Val Acc: 0.868
Epoch: 6, BCELoss: 0.18482356718430917
Val Loss: 0.350 	Val Acc: 0.871
Epoch: 7, BCELoss: 0.1674177346068124
Val Loss: 0.368 	Val Acc: 0.868
Epoch: 8, BCELoss: 0.1564461703443279
Val Loss: 0.391 	Val Acc: 0.861
Epoch: 9, BCELoss: 0.142950885038202


[I 2022-05-14 21:38:07,263] Trial 14 finished with value: 84.07 and parameters: {'optimizer': 'Adam', 'lr': 0.010157904172026881, 'dropout': 0.44086722785937166, 'hidden_size': 71, 'embed_size': 62, 'filter_size': 143}. Best is trial 2 with value: 87.81.


Val Loss: 0.485 	Val Acc: 0.841
Study statistics: 
  Number of finished trials:  15
  Number of complete trials:  15
Best trial:
  Value:  87.81
  Params: 
    optimizer: Adam
    lr: 0.009127263281980543
    dropout: 0.36742034519123545
    hidden_size: 117
    embed_size: 31
    filter_size: 103


In [17]:
torch.save(best_model.state_dict(),"./cnn-lstm-imdb-hyperband.pth")

In [18]:
test(best_model, test_dl, epochs)

/ext3/miniconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:691: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:925.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Accuracy: 88.03999999999999
